<a href="https://colab.research.google.com/github/nisaodabas/CNG483-Project2/blob/master/CNG483_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [270]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from google.colab import output
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def createDataSet(path):
    df = pd.read_fwf(path)
    data = np.array(df)

    numAttr = 0
    rows = []
    for row in data:        
        found = row[0].find("@ATTRIBUTE")
        if(found != -1):
            numAttr += 1
        else:
            # ignore @ATTRIBUTE rows
            rows.append(row[0].split(","))

    # delete @DATA row
    rows = np.array(rows[1:])
    
    labels = rows[:, -1].astype('int32')
    features = rows[:, :numAttr-1].astype('float32') 

    return features, labels

In [0]:
def model():
    # determine the number of input features
    #n_features = X_train.shape[1]
    # define model
    model = Sequential()
    model.add(Dense(10, activation='relu', kernel_initializer='he_normal', input_dim=5))
    model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(3, activation='softmax'))
    # compile the model
    #opt = SGD(learning_rate=0.01, momentum=0.9)
    #model.compile(optimizer=opt, loss='binary_crossentropy')
    model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model



    '''
    # fit the model
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    # evaluate the model
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print('Test Accuracy: %.3f' % acc)
    '''
    

In [0]:
if __name__ == "__main__":
    geometic_test_path = "/content/drive/My Drive/CNG483-Project 2/IrisGeometicFeatures_TestingSet.txt"
    geometic_train_path = "/content/drive/My Drive/CNG483-Project 2/IrisGeometicFeatures_TrainingSet.txt"

    texture_test_path = "/content/drive/My Drive/CNG483-Project 2/IrisTextureFeatures_TestingSet.txt"
    texture_train_path = "/content/drive/My Drive/CNG483-Project 2/IrisTextureFeatures_TrainingSet.txt"


    geo_x_train, geo_y_train = createDataSet(geometic_train_path)
    geo_x_test, geo_y_test = createDataSet(geometic_test_path)


In [0]:
    #model(geo_x_train, geo_y_train, geo_x_test, geo_y_test)
    for i in range(25,50,5):

        estimator = KerasClassifier(build_fn=model, epochs=300, batch_size=32, verbose=0)
        kfold = KFold(n_splits=10, shuffle=True, random_state=30)
        results = cross_val_score(estimator, geo_x_train, geo_y_train, cv=kfold, scoring='accuracy')
        print("rand = ", i)
        print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

rand =  25
Baseline: 49.56% (10.59%)
rand =  30
Baseline: 50.07% (12.57%)
rand =  35
Baseline: 50.86% (11.48%)
rand =  40
Baseline: 40.69% (18.94%)


In [0]:
#output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')